In [ ]:
import os
from src.analyzer import *
%matplotlib inline
output_dir = 'output/sparsity'

In [ ]:
pkl_fns = [os.path.join(output_dir, fn) 
           for fn in os.listdir(output_dir) 
           if fn.endswith('.pkl')]
pkl_fns

In [ ]:
snr_list_ = [DataAnalyzer.fromfilename(pkl_fn).SNR_list() for pkl_fn in pkl_fns]

In [ ]:
in_idx = slice(0, 1)
sp_idx = slice(1, 2)

In [ ]:
da = DataAnalyzer.fromfilename(pkl_fns[-1])

In [ ]:
snr_sp = np.array(snr_list_[sp_idx])
snr_id = np.array(snr_list_[in_idx])
t = da.time_list()

In [ ]:
def plot_fill_between(t, data, label='', c = 'r'):
    """
    t : array, shape (timesteps, )
        indices for time
    data : array, shape (samples, timesteps)
        SNR to plot mean and +/- one sdev
    """
    mm = data.mean(0)
    sd = data.std(0)
    plt.fill_between(t, mm-sd, mm + sd, alpha=0.5, color=c)
    plt.plot(t, mm, color=c, label=label)

In [ ]:
plot_fill_between(t, snr_sp, c = 'g', label='Sparse Prior')
plot_fill_between(t, snr_id, c = 'b', label='Independent Pixel Prior')
plt.xlabel('time (ms)')
plt.ylabel('SNR')
plt.title('SNR as a function of time, DC = {}'.format(100.))
plt.legend(loc='upper left')
plt.savefig(os.path.join(output_dir, 'sp_vs_indep.png'), dpi=200)